In [35]:
import numpy as np
import pandas as pd


In [45]:
all_games = pd.read_csv('csv/games.csv', header=None)

# for i in [0, 2]:
#     all_games[i][all_games[i] == 'New Jersey Nets'] = 'Brooklyn Nets'
#     all_games[i][all_games[i] == 'Charlotte Bobcats'] = 'Charlotte Hornets'
#     all_games[i][all_games[i] == 'New Orleans Hornets'] = 'New Orleans Pelicans'
#all_games.to_csv('csv/games.csv', index=None, header=False)

total_wins = {}
for year in xrange(2011, 2015):
    wins = {}
    games = all_games[all_games[4] == year]
    for index, game in games.iterrows():
        if game[3] > game[1]:
            if game[2] not in wins:
                wins[game[2]] = 1
            else:
                wins[game[2]] += 1
        else:
            if game[0] not in wins:
                wins[game[0]] = 1
            else:
                wins[game[0]] += 1
    total_wins[year] = wins

In [39]:
team_stats = {}
for year in range(2011, 2017):
    off = pd.read_csv('csv/team_stats/off/'+str(year)+'.csv').sort_values(by=['Team'])
    off.columns = ['Off'+col if col!='Team' else col for col in off.columns]
    defense = pd.read_csv('csv/team_stats/def/'+str(year)+'.csv').sort_values(['Team'])
    defense.columns = ['Def'+col for col in defense.columns]
    off_col = [off.columns.values[1]]+list(off.columns.values[4:])
    def_col = list(defense.columns.values)[4:]
    team_stats[year] = pd.concat([off[off_col], defense[def_col]], axis=1)
team_stat_names = list(team_stats[2011].columns.values)[1:]
opp_stat_names = ['Opp'+name for name in team_stat_names]
column_names = team_stat_names + opp_stat_names
column_names.append('YrsSince2011')
column_names.append('Home')
column_names.append('OffWins')
column_names.append('DefWins')

In [40]:
def make_x(train_or_test):
    data = pd.read_csv('games_'+train_or_test+'.csv', delimiter=',', header=None)
    features = pd.DataFrame(index=range(data.shape[0]*2), columns = column_names)
    for data_ind, game in data.iterrows():
        year = game[4]
        away_team = game[0]
        home_team = game[2]
        away_wins = total_wins[year - 1][away_team]
        home_wins = total_wins[year - 1][home_team]
        for team_ind,team in team_stats[year-1].iterrows():
            if team['Team']==away_team:
                away_team_stats=team[team_stat_names]
            if team['Team']==home_team:
                home_team_stats=team[team_stat_names]
        away_row = np.concatenate((away_team_stats.append(home_team_stats).as_matrix(),[year-2011],[0],[away_wins],[home_wins]))
        home_row = np.concatenate((home_team_stats.append(away_team_stats).as_matrix(),[year-2011],[1],[home_wins],[away_wins]))
        features.loc[2*data_ind] = away_row
        features.loc[2*data_ind+1] = home_row
    features.to_csv('csv/x_'+train_or_test+'.csv',index=None)
make_x('train')
make_x('test')

KeyError: 'Charlotte Bobcats'